# Watchlist example notebook

```Author: Francisco Förster, Last update: 20251009```

Here we will do a xmatch against a list of positions in the sky, given in the file ```watchlist.csv```.

*It is highly recommended that you try this notebook in Google Colab using the following [link](https://colab.research.google.com/github/alercebroker/usecases/blob/master/notebooks/ALeRCE_Other_Watchlist.ipynb).*
This will avoid you from having to sort out library installation problems and focus on the contents of the tutorial. You can try installing the dependencies later in your own system.

In [1]:
import pandas as pd
import sqlalchemy as sa
import requests

# Get credentials to open a direct connection with the database

In [2]:
credentials_file = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(credentials_file).json()["params"]

In [3]:
engine = sa.create_engine("postgresql+psycopg2://" + params["user"] \
                          + ":" + params["password"] + "@" + params["host"] \
                          + "/" + params["dbname"])
conn = engine.connect()

# Get watchlist of object positions

In [4]:
df = pd.read_csv("https://github.com/alercebroker/usecases/blob/master/example_data/watchlist.csv?raw=True")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


# Prepare the query string for watchlist for object within 1 degress of given positions, and  happening within the last 10 days

In [5]:
objects = []
for _,row in df.iterrows():
    objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
objects_str = ",\n".join(objects)
objects_str

"('source_1', 160.18301441363647, 33.0164673528409),\n('source_2', 174.21524897555545, 44.83789535222221)"

Create query string

In [6]:
from astropy.time import Time
nt = Time.now()
last_mjd_discovery = nt.mjd - 10

In [7]:
search_radius = 1 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
)
SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec,
    q3c_dist(c.ra, c.dec, o.meanra, o.meandec), o.firstmjd
FROM object o, catalog c
    /*
       It is REALLY important to first use the catalog then the object
       ra, dec for speed. The radius is in degrees.
    */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
""" % (last_mjd_discovery)

# Radius in degrees
query_str = query.format(values=objects_str, radius=search_radius)

In [8]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(59, 8)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd
0,source_1,160.183014,33.016467,ZTF25abuqctv,159.069988,32.868116,0.945775,60948.528657
1,source_1,160.183014,33.016467,ZTF25abuqdpw,159.813900,32.523936,0.582164,60948.533750
2,source_1,160.183014,33.016467,ZTF18adrkmuj,160.712072,32.343574,0.806894,60948.533750
3,source_1,160.183014,33.016467,ZTF25abuqelh,159.074232,32.881377,0.940192,60948.538843
4,source_1,160.183014,33.016467,ZTF18adrkdxh,159.082354,32.932633,0.927151,60948.538843
5,source_1,160.183014,33.016467,ZTF18adrkdxd,159.242244,33.226192,0.815341,60948.538843
6,source_1,160.183014,33.016467,ZTF19adfwokp,160.227050,32.099065,0.918153,60948.538843
7,source_1,160.183014,33.016467,ZTF18acsvgbz,161.176668,33.342140,0.893138,60948.538843
8,source_1,160.183014,33.016467,ZTF18adrkdvf,160.195604,32.029194,0.987331,60948.538843
9,source_1,160.183014,33.016467,ZTF25abuqfru,160.637528,33.232510,0.437684,60948.538843


# Query objects within 10 degress of given positions, first detected within the last 10 days, with SN probabilities > 0.4, and ranking=1 in the stamp classifier

In [9]:
search_radius = 10 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
),
sn (oid, classifier_name, class_name, probability, ranking) AS (
    SELECT
        o.oid, p.classifier_name, p.class_name, p.probability, p.ranking
    FROM
        probability p
    INNER JOIN 
        object o
    ON 
        o.oid=p.oid
    WHERE
        p.classifier_name = 'stamp_classifier'
        AND p.class_name IN ('SN')
        AND p.ranking = 1
        AND p.probability > 0.4
        AND o.firstmjd > {last_mjd_discovery}
)
SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec,
    q3c_dist(c.ra, c.dec, o.meanra, o.meandec), o.firstmjd,
    sn.classifier_name, sn.class_name, sn.probability, sn.ranking
FROM object o INNER JOIN sn ON sn.oid = o.oid, catalog c
    /*
       It is REALLY important to first use the catalog then the object
       ra, dec for speed. The radius is in degrees.
    */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > {last_mjd_discovery}
ORDER BY
    c.source_id
"""
# Radius in degrees
query_str = query.format(values=objects_str, radius=search_radius,
                         last_mjd_discovery=last_mjd_discovery)

In [10]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches.head(100)

(3, 12)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd,classifier_name,class_name,probability,ranking
0,source_1,160.183014,33.016467,ZTF25abuqccu,155.394065,23.958388,9.984662,60948.529618,stamp_classifier,SN,0.567389,1
1,source_1,160.183014,33.016467,ZTF25abuynvz,152.229680,35.846430,7.141306,60949.523380,stamp_classifier,SN,0.753734,1
2,source_1,160.183014,33.016467,ZTF25abwbvnx,169.451244,31.465964,7.988117,60955.532882,stamp_classifier,SN,0.459108,1


Close the connection

In [11]:
conn.close()

# Open the resulting objects in the ALeRCE explorer

In [12]:
url = "https://alerce.online/?" \
      + "&".join(["oid=%s" % oid for oid in matches.oid.values]) \
      + "&count=true&page=1&perPage=1000&sortDesc=false"
print(url)

https://alerce.online/?oid=ZTF25abuqccu&oid=ZTF25abuynvz&oid=ZTF25abwbvnx&count=true&page=1&perPage=1000&sortDesc=false
